In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

In [15]:
# get list of files in location and construct list of files through which to loop

mypath = '/Users/ksipes/Documents/Scripts/findHQfa'
file_list = pd.Series(data=[f for f in listdir(mypath) if isfile(join(mypath, f))])
files = pd.DataFrame.from_records(data = file_list.str.split('.'), columns = ['name', 'extension'])
files['full_name'] = files.name + '.' + files.extension

print('list of all files...')
print(files)
print('\n')

loop_files = files[files.extension == 'tsv'].full_name

print('all files through which to loop...')
print(loop_files)

list of all files...
                       name extension                     full_name
0                            DS_Store                     .DS_Store
1                 B3_SPAdes       tsv                 B3_SPAdes.tsv
2  enzyme_counts_to_culture       csv  enzyme_counts_to_culture.csv
3                G19_SPAdes       tsv                G19_SPAdes.tsv
4                 E7_SPAdes       tsv                 E7_SPAdes.tsv
5                 B4_SPAdes       tsv                 B4_SPAdes.tsv
6    assemblyqualitymarkers      xlsx   assemblyqualitymarkers.xlsx


all files through which to loop...
1     B3_SPAdes.tsv
3    G19_SPAdes.tsv
4     E7_SPAdes.tsv
5     B4_SPAdes.tsv
Name: full_name, dtype: object


In [16]:
# Instantiate master_df, then loop through loop_files, adding to the master_df and including organism field
master_df = pd.DataFrame()

for f in loop_files:
    organism_name = f.split('_')[0]
    print('reading in {}...'.format(f))
    df = pd.read_csv('findHQfa/' + f, sep='\t')
    df['organism'] = organism_name
    print('shape of resulting DataFrame for {} is {}'.format(f, df.shape))
    master_df = master_df.append(df)

print('\nfinal shape of master DataFrame is {}'.format(master_df.shape))

for c in ['gene', 'product']:
    master_df[c] = master_df[c].str.lower()

reading in B3_SPAdes.tsv...
shape of resulting DataFrame for B3_SPAdes.tsv is (11161, 8)
reading in G19_SPAdes.tsv...
shape of resulting DataFrame for G19_SPAdes.tsv is (19811, 8)
reading in E7_SPAdes.tsv...
shape of resulting DataFrame for E7_SPAdes.tsv is (13179, 8)
reading in B4_SPAdes.tsv...
shape of resulting DataFrame for B4_SPAdes.tsv is (5853, 8)

final shape of master DataFrame is (50004, 8)


In [9]:
#read in excel file containing gene to enzyme mapping
temp_df = pd.read_excel('/Users/ksipes/Documents/Scripts/findHQfa/assemblyqualitymarkers.xlsx')

for c in temp_df.columns:
    temp_df[c] = temp_df[c].str.lower()

gene_to_enzyme = temp_df.set_index('gene').to_dict()['Category']

In [11]:
#map columns to enzymes
master_df['product'] = [gene_to_enzyme.get(prod) for prod in master_df['product']]
master_df['gene'] = [gene_to_enzyme.get(gene) for gene in master_df['gene']]

#find good way to combine
#create final enzyme column
master_df['final'] = master_df.product

In [14]:
#export final enzyme counts for each organism to csv
gene_counts = master_df[['organism', 'final']].pivot_table(index='organism', columns = 'final', aggfunc=len)

gene_counts.to_csv('/Users/ksipes/Documents/Scripts/findHQfa/enzyme_counts_to_culture.csv')
master_df.to_csv('/Users/ksipes/Documents/Scripts/findHQfa/master_output.csv')

master_df.final.value_counts()

RecursionError: maximum recursion depth exceeded while calling a Python object